unzip data

https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data

In [20]:
!file /content/all_data.csv.zip

/content/all_data.csv.zip: Zip archive data, at least v4.5 to extract, compression method=deflate


In [21]:
!ls -lh /content/all_data.csv.zip


-rw-r--r-- 1 root root 327M Jan 12 11:42 /content/all_data.csv.zip


In [22]:
!7z x /content/all_data.csv.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 341878979 bytes (327 MiB)

Extracting archive: /content/all_data.csv.zip
--
Path = /content/all_data.csv.zip
Type = zip
Physical Size = 341878979

  0%      0% - all_data.csv                     1% - all_data.csv                     2% - all_data.csv                     3% - all_data.csv                     4% - all_data.csv                     6% - all_data.csv                     8% - all_data.csv                   

let's explore the data

In [1]:
pip install num2words

In [2]:
import pandas as pd
import numpy as np
import re
import html
from num2words import num2words
import string
import unicodedata
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('/content/all_data.csv')

In [4]:
df.head(5)

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [5]:
df.columns

Index(['id', 'comment_text', 'split', 'created_date', 'publication_id',
       'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes',
       'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'identity_annotator_count',
       'toxicity_annotator_count'],
      dtype='object')

In [6]:
unique_values,count = np.unique(df['split'], return_counts=True)
print(unique_values)
print(count)

['test' 'train']
[ 194641 1804875]


In [7]:
df['comment_text'][0]

"He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed."

In [8]:
unique_values,count = np.unique(df['rating'], return_counts=True)
print(unique_values)
print(count)

['approved' 'rejected']
[1867134  132382]


lets do some operation on data before text preprocessing

In [9]:
train_texts = df[df['split'] == 'train']['comment_text'].values
test_texts = df[df['split'] == 'test']['comment_text'].values

train_labels = df[df['split'] == 'train']['toxicity'].values
test_labels = df[df['split'] == 'test']['toxicity'].values


In [10]:
train_labels = np.asarray(train_labels>=0.5).astype('float32')
test_labels = np.asarray(test_labels>=0.5).astype('float32')

In [11]:
train_texts[0]

"He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed."

In [12]:
train_texts.shape

(300000,)

In [13]:
train_labels[0]

np.float32(0.0)

In [14]:
train_labels.shape

(300000,)

# text preprocessing

In [15]:
def remove_specific_chars(text):
  re1 = re.compile(r'  +')
  x = text.replace('#39;', "").replace('amp;', '').replace('#146;', "").replace(
      'nbsp;', '').replace('#36;', '').replace('\\n', "").replace('quot;', "").replace(
      '<br />', "").replace('\\"', '').replace('<unk>','').replace(' @.@ ','').replace(
      ' @-@ ','').replace('\\', '')
  return re1.sub(' ',html.unescape(x))


def remove_non_ascii(text):
  return unicodedata.normalize('NFKD' ,text).encode('ascii' ,'ignore').decode('utf8' ,'ignore')


def to_lowercase(text):
  return text.lower()


def remove_punctuation(text):
  translator = str.maketrans('','',string.punctuation)
  return text.translate(translator)


def replace_numbers(text):
  return re.sub(r'\d+' , lambda x: num2words(int(x.group())) ,text)


def remove_whitespaces(text):
  return text.strip()


def remove_stopwords(words,stop_words):
  return [word for word in words if word not in stop_words]


def lemmatizer_verbs(words):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(w , pos = 'v') for w in words]


def text2words(text):
  return word_tokenize(text)



STOP_WORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

def normalize_text(text):

  if not isinstance(text,str):
      return ""
  stop_words = stopwords.words('english')
  text = remove_specific_chars(text)
  text = remove_non_ascii(text)
  text = remove_punctuation(text)
  text = to_lowercase(text)
  text = replace_numbers(text)
  text = remove_whitespaces(text)

  words = text2words(text)
  words = [w for w in words if w not in STOP_WORDS]
  words = [LEMMATIZER.lemmatize(w, pos='v') for w in words]

  return ' '.join(words)


Now let's apply this on the whole corpus

In [16]:
def normlize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [17]:
norm_train_texts =  normlize_corpus(train_texts)
norm_test_texts =  normlize_corpus(test_texts)

In [18]:
# def normalize_in_batches(texts, batch_size=50000):
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i+batch_size]
#         yield [normalize_text(t) for t in batch]

In [19]:
# norm_train_texts = []

# for batch in normalize_in_batches(train_texts):
#     norm_train_texts.extend(batch)

# norm_test_texts = []

# for batch in normalize_in_batches(test_texts):
#     norm_test_texts.extend(batch)


In [20]:
print(norm_train_texts[0])
print(train_texts[0])

get money lie wait till election two yrs dirty politicians need afraid tar feather arent people get screw
He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed.


# text preperation
we will use keras tokenizer

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [22]:
text = norm_train_texts+norm_test_texts

In [23]:
tok = Tokenizer(oov_token = 'UNK' , num_words = 10000)
tok.fit_on_texts(text)


Tokenizer can also be used for few more things:

word counts: A dictionary of words and their counts.

word docs: An integer count of the total number of documents that were used to fit the Tokenizer.

word index: A dictionary of words and their uniquely assigned integers.

document count: A dictionary of words and how many documents each appeared in.

In [24]:
l = tok.texts_to_sequences(norm_train_texts[0].split())
l

[[3],
 [37],
 [208],
 [295],
 [2446],
 [200],
 [12],
 [3983],
 [1860],
 [495],
 [19],
 [1487],
 [4122],
 [5162],
 [377],
 [5],
 [3],
 [1417]]

In [25]:
tok.sequences_to_texts(l)

['get',
 'money',
 'lie',
 'wait',
 'till',
 'election',
 'two',
 'yrs',
 'dirty',
 'politicians',
 'need',
 'afraid',
 'tar',
 'feather',
 'arent',
 'people',
 'get',
 'screw']

In [26]:
x_train = tok.texts_to_matrix(norm_train_texts , mode = 'freq')
x_test = tok.texts_to_matrix(norm_test_texts , mode = 'freq')

In [27]:
print(x_train.shape)
print(x_test.shape)
print(train_labels.shape)
print(test_labels.shape)


(300000, 10000)
(50000, 10000)
(300000,)
(50000,)


In [ ]:
# from tensorflow.keras.layers import TextVectorization

# vectorizer = TextVectorization(
#     max_tokens=10000,
#     output_mode="tf_idf",   # أو "binary" أو "tf-idf"
#     sparse=True
# )

# vectorizer.adapt(norm_train_texts)


In [ ]:
# import tensorflow as tf

# train_ds = tf.data.Dataset.from_tensor_slices((norm_train_texts, y_train))
# train_ds = (
#     train_ds
#     .shuffle(buffer_size=50000)
#     .batch(512)
#     .map(lambda x, y: (vectorizer(x), y))
#     .prefetch(tf.data.AUTOTUNE)
# )

# val_ds = tf.data.Dataset.from_tensor_slices((norm_test_texts, y_test))
# val_ds = (
#     val_ds
#     .batch(512)
#     .map(lambda x, y: (vectorizer(x), y))
#     .prefetch(tf.data.AUTOTUNE)
# )



# model